# Hands-on AWS - 2 - SQS pipeline

## Installing dependencies
Here we install relevant dependencies to run serverless framework.

In [ ]:
!pip install awscli --upgrade --user
!npm install -g serverless
!echo -e '\n\033[1mStep finished\033[0m'

## Setting AWS environmental variables
Here we set up AWS environmental variables so that we will be able to deploy to our AWS account. We will need access key id, secret access key and account id. You will need to replace AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_ACCOUNT_ID with your values. Please use test account and temporary credentials or deactivate credentials after usage.

In [ ]:
%env AWS_ACCESS_KEY_ID=<AWS_ACCESS_KEY_ID>
%env AWS_SECRET_ACCESS_KEY=<AWS_SECRET_ACCESS_KEY>
%env AWS_ACCOUNT_ID=<AWS_ACCOUNT_ID>
%env AWS_DEFAULT_REGION=us-west-1

## Setting up identifier
Here we set up personal identifier. You will need to replace "FirstName LastName" with your first name and lastname.

In [ ]:
import hashlib
import datetime
res=hashlib.sha224(bytes("FirstName LastName"+str(datetime.datetime.now().timestamp()),'utf-8')).hexdigest()[:5]
%env HASH=$res

## SQS+Lambda deployment

Now let's deploy the lambda with SQS queues

In [ ]:
!cd hands-on-aws-2-sqs-pipeline;npm install
!cd hands-on-aws-2-sqs-pipeline;serverless deploy --verbose > logs.txt
!echo -e '\n\033[1mStep finished\033[0m'

Let's view the output of the deployment process. Serverless framework will return the queues which we created

In [ ]:
!cd hands-on-aws-2-sqs-pipeline;tail -15 logs.txt
!echo -e '\n\033[1mStep finished\033[0m'

## Sending message to SQS and looking at the logs

Let's send simple message to the SQS and check the it's read by the lambda.

In [ ]:
!aws sqs send-message --queue-url $(cd hands-on-aws-2-sqs-pipeline;tail -15 logs.txt | grep "ReadQueueUrl: " | cut -f2- -d:) --message-body "HelloWorld"
!echo -e '\n\033[1mStep finished\033[0m'

Now let's take a look at the latest logs for the function and check that it received the message.

In [ ]:
!sleep 10
!cd hands-on-aws-2-sqs-pipeline;serverless logs -f HelloWorld
!echo -e '\n\033[1mStep finished\033[0m'

Let's check that lambda wrote the message to S3 by checking message id key.

In [ ]:
!aws s3 cp --quiet s3://vhive-aws-tutorial/$(cd hands-on-aws-2-sqs-pipeline;serverless logs -f HelloWorld | grep -m 1 "Message id:" | cut -f2- -d:) /dev/stdout
!echo -e '\n\033[1mStep finished\033[0m'

Now let's send a message which will result in error on lambda side.

In [ ]:
!aws sqs send-message --queue-url $(cd hands-on-aws-2-sqs-pipeline;tail -15 logs.txt | grep "ReadQueueUrl: " | cut -f2- -d:) --message-body "ValueError"
!echo -e '\n\033[1mStep finished\033[0m'

Let's check how this message will be read by lambda.

In [ ]:
!sleep 10
!cd hands-on-aws-2-sqs-pipeline;serverless logs -f HelloWorld
!echo -e '\n\033[1mStep finished\033[0m'

Since the lambda failed the message will be resent to the dead letter queue.

In [ ]:
!sleep 10
!aws sqs receive-message --queue-url $(cd hands-on-aws-2-sqs-pipeline;tail -15 logs.txt | grep "DeadLetterQueueUrl: " | cut -f2- -d:)
!echo -e '\n\033[1mStep finished\033[0m'

## Remove infrastructure

Now let's do the cleanup process and remove the infrastructure which we created.

In [ ]:
!cd hands-on-aws-2-sqs-pipeline;serverless remove > logs.txt
!echo -e '\n\033[1mStep finished\033[0m'